In [5]:
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from collections import Counter
from gensim.models import word2vec

## Text processing

In [6]:
df_authors = pd.read_csv('AllAuthors.csv')
df_authors = df_authors.dropna()
df_authors

,userName,displayName,description
1,leungkaichihk,梁啟智,副業是在香港中文大學教書，主業是玩貓。
2,hymnsin,Hymnsin,Matters 早期种子用户 |\n\n渴望真理，保持冷静 |\n\n现实系小说《洋葱人》U...
3,iamiyabi,LiyaChen,台北貓奴。人在上海
4,disincurable,方可成,宾夕法尼亚大学传播学博士候选人
5,lincalvino,寓森,林寓森，精神科醫師，喜歡思考與寫作，愛好騎單車。目前主要關注「自戀」與「無條件基本收入」的主...
6,sophia,黄雪琴,黄雪琴，独立记者，公民，关注性别、平权、弱势群体
7,baize417,Pasqual,关注知识产权、区块链、二次元文化、自闭谱系障碍。
8,levisliqiuwang,王立秋,译者，关注伊斯兰研究。
9,yt,小月,暫時還不想暴露身份的 Matters 團隊成員，一隻蟄伏的 PM
10,brucelai,賴勇衡,一臉書塵鼻敏感，半室映畫眼昏昏。\n電影及戲劇評論人，am730專欄「超齡學生大笨鐘」作者，...


In [7]:
df_news = pd.read_csv('NewestFeed.csv')
df_news = df_news[['slug', 'createdAt', 'summary', 'author_userName', 'author_displayName','MAT']]
df_news.rename(columns={'slug': 'title', 
                        'author_userName': 'userName', 
                        'author_displayName': 'displayName'}, 
               inplace=True)

In [13]:
# MAT filtered
df_news = df_news[df_news['MAT'] != 0]
df_news = df_news.dropna(subset = ['MAT'])
df_news.sort_values(by=['MAT'], ascending=False, inplace=True)

In [14]:
# cut words in titles
df_news['title'] = df_news['title'].astype(str)

def cutwords(x):
    segments = jieba.analyse.extract_tags(x, topK=20, withWeight=False, allowPOS=())
    seg_list = []
    for seg in segments:
        if str(seg) != '-':
            if str(seg).isnumeric() is False: 
                 seg_list.append(seg)
    return ' '.join(seg_list)

df_news['seg_words'] = df_news['title'].apply(cutwords)

In [15]:
# trim date
df_news['createdAt'] = df_news['createdAt'].astype(str)

def trim_date(x):
    y = x[0:10]
    return y

df_news['createdAt'] = df_news['createdAt'].apply(trim_date)
df_news

,title,createdAt,summary,userName,displayName,MAT,seg_words
5063,我也來一起洗mat吧,2018-05-22,看了 @吳秉儒 的貼文，發現洗錢好像很好玩，所以我也來試試看～～ 原貼文如下： ------...,voicelee,voicelee,5448.0,mat 一起
5105,一起來洗mat吧,2018-05-18,大家好，長話短說， 今天Matters團隊公布了一個小改版，讓點讚的功能變得像Medium拍...,tomwu,吳秉儒,2000.0,mat 一起
5415,matters日記1-matters-從問對問題開始,2018-04-08,在Matters之前，我做過12年新聞人。做過新聞週刊的記者，像書一樣策劃專題的月刊副主編，...,az,潔平,797.0,matters 日記 從問 問題 開始
5569,个人微信号被封记-信息难民何去何从-兼问各位,2018-03-05,必须承认，发现自己的个人微信号被封时，我有点发懵。2月27日清早起来，我习惯性地打开手机，想...,freedom6205,江雪,754.0,封记 兼问 何去何从 难民 各位 信号 个人 信息
5482,我為何像恐怖舊情人一樣追殺臉書,2018-03-21,前言：多謝@ckxpress 及 @映昕 的經驗，身為一名逃離臉書及Line的老孤怪，深有同...,danderh,黃哲斌,718.0,我為 何像 一樣 追殺 臉書 情人 恐怖
5553,美國右翼運動觀察,2018-03-08,我個人主要做進步社運的研究，這兩年才開始關注歐美右翼，一是因為覺得學界對這個話題的討論太少了...,zoezhao,夕岸,695.0,美國 運動 觀察 右翼
5735,软掉的非虚构-瞎掉的舆论场-非虚构写作讨论-下,2018-02-01,以上做一个整体的勾勒，下面简单讲几点感受。非虚构的虚构问题。这个问题，主要是真实故事计划和人...,zhuyu3,朱玉,679.0,虚构 瞎掉 软掉 写作 舆论 讨论
5545,我觉得我做的事很温和-但不代表它不是一种争夺,2018-03-09,□■□ 2 月 25 日那天，我觉得非常丧，于是就在微信公众号上写了一篇心情笔记发了。最后一...,rainfieldfifi,F小姐,663.0,争夺 温和 代表 觉得 一种 不是
5497,對於-im-的厭倦,2018-03-19,這篇實在讓我心有戚戚焉，昨天半夜在熬夜趕稿，本來想要馬上回覆，但還是先忍耐住，今天截完了一定...,yingshinlee,映昕,656.0,im 厭倦
5525,萬有霍金,2018-03-14,他曾很驕傲自己是在伽利略誕辰300週年出生，如今，他也會感到榮耀，在愛因斯坦誕辰的紀念日逝世...,chaiziwen,柴子文,648.0,霍金 萬有


In [16]:
df = pd.merge(df_authors, df_news, on= 'userName', how = 'outer')
# df = pd.merge(df_authors, df_news, left_on = 'userName', right_on = 'displayName', how = 'outer')
df.to_csv('comprehensive.csv')

In [17]:
df

,userName,displayName_x,description,title,createdAt,summary,displayName_y,MAT,seg_words
0,leungkaichihk,梁啟智,副業是在香港中文大學教書，主業是玩貓。,因為想超越-蛇齋餅糭-所以想到做社區報,2018-01-25,謝謝邀請分享。要談社區報和民主日常之前，得先繞一段路，談一談香港的選舉生態。香港最為人關心的...,梁啟智,636.0,因為 社區 超越 想到 所以
1,leungkaichihk,梁啟智,副業是在香港中文大學教書，主業是玩貓。,如果社交網絡是沙灘-什麼是洋流,2018-04-10,潔平出來說話了，我也說兩句。很早便獲邀請加入 matters 的內測，一直對這事情不敢樂觀，...,梁啟智,240.0,網絡 沙灘 什麼 洋流 社交 如果
2,leungkaichihk,梁啟智,副業是在香港中文大學教書，主業是玩貓。,反送中答問集,2019-06-12,前言：二零一四年九月二十八日，我在淚水中寫了〈香港怎麼了：佔中十七問〉，協助中國大陸和台灣讀...,梁啟智,69.0,反送 中答 問集
3,leungkaichihk,梁啟智,副業是在香港中文大學教書，主業是玩貓。,香港第一課-簡介及目錄,2019-05-13,簡介這本來是我在香港中文大學教書時，寫給內地生的課堂筆記。現時每年有約兩萬名中國大陸學生前來...,梁啟智,44.0,簡介及 目錄 香港 第一
4,leungkaichihk,梁啟智,副業是在香港中文大學教書，主業是玩貓。,香港沒有暴亂,2019-06-16,我本來想過再寫一篇答客問，澄清這兩天中國大陸媒體對香港的「暴動」抹黑。但想來想去，發現事情沒...,梁啟智,43.0,沒有 暴亂 香港
5,leungkaichihk,梁啟智,副業是在香港中文大學教書，主業是玩貓。,香港第一課-14-中港對特區政治制度的最大分歧是什麼,2019-05-27,中國政府和香港社會對於特區政府的認授來源有根本分歧，香港各種政制爭議歸根究底均由此而來。特區...,梁啟智,32.0,特區 什麼 中港 分歧 香港 第一 制度 政治 最大
6,leungkaichihk,梁啟智,副業是在香港中文大學教書，主業是玩貓。,香港第一課-前言-答問作為一種異議,2019-05-13,這個寫作計劃的緣起，要說到八年前獲香港中文大學新聞與傳播學院邀請，我在他們的碩士班開課講香港...,梁啟智,28.0,答問 作為 一種 異議 前言 香港 第一
7,leungkaichihk,梁啟智,副業是在香港中文大學教書，主業是玩貓。,香港第一課-1-香港自古以來不就是中國領土嗎,2019-05-14,回答問題前，總得先解題。這條問題的出現，往往是以反問句而非疑問句的方式，發問者對此已有答案，...,梁啟智,26.0,以來 中國 領土 香港 自古 第一 就是
8,leungkaichihk,梁啟智,副業是在香港中文大學教書，主業是玩貓。,香港第一課-5-香港認同和中國認同是否對立,2019-05-18,香港認同和中國認同本來不一定互相排斥。在香港歷史的絕大多數時間中，香港認同和中國認同有著微妙...,梁啟智,23.0,認同 中國 對立 香港 是否 第一
9,leungkaichihk,梁啟智,副業是在香港中文大學教書，主業是玩貓。,香港第一課-3-中國大陸誤解香港帶來什麼後果,2019-05-16,誤解往往帶來矛盾和對立。但在討論負面後果前，先說一些常常被忽略的，來自誤解的「正面後果」。九...,梁啟智,22.0,中國大陸 誤解 帶來 什麼後果 香港 第一


## author2keywords

In [19]:
# create dictionary
author2kewords = {}
for i in df_news['displayName'].unique():
    temp = []
    for j in df_news[df_news['displayName']==i].index:
        temp += df_news['seg_words'][j].split(' ')
        temp = list(set(temp))
    author2keywords[i] = temp
    
author2keywords

{'voicelee': ['一起', 'mat'],
 '吳秉儒': ['做到',
  '一起',
  '體系',
  '到底',
  '分享',
  '運作',
  'ai',
  '建議',
  '什麼',
  'mat',
  '洗錢',
  'ptt',
  '必須',
  '平台',
  '這幾天',
  '區塊',
  '一點',
  '目前',
  '思考',
  '體系前',
  '看法',
  '模式',
  '權重',
  'matters',
  '心得',
  '可能',
  '可以',
  '生態'],
 '潔平': ['今天',
  '转载',
  '何去',
  '談科學',
  '這篇',
  '北京',
  '回到',
  '绿皮',
  '守门人',
  '戰略',
  '比技藝',
  '港澳',
  '怎麼',
  '一封信',
  '劉蘇里',
  '介紹',
  '朋友',
  '六年',
  '两家',
  '一個',
  '告别',
  '尋找',
  '持續',
  '陌生人',
  '兇險',
  '時代',
  '往事',
  '看過',
  '所有',
  '之前',
  '局勢',
  '称为',
  '之一',
  '八旗',
  '信息',
  '李宗盛',
  'taxi',
  '襲警',
  '還能',
  '台大',
  '核试验',
  '一天',
  '反送',
  '以前',
  '广场',
  '獨立',
  '國家',
  '小时',
  'newsroom',
  '那些',
  '江雪',
  '學到',
  '墮落',
  '之間',
  '新聞',
  '哲學',
  '偶遇',
  '蒼白',
  '重要',
  '號角',
  '北大',
  '筆記',
  '意味著',
  '抗命',
  '火车',
  '中運動',
  '见文',
  '因何',
  '岳昕',
  '文摘',
  '三角地',
  '升級',
  '每个',
  '預示',
  '金正恩',
  '滿字',
  '合輯',
  '元朗',
  '還認',
  '下的粵',
  '空間',
  '書摘',
  '失敗',
  '變化',
  '需要',
  '李志德',
  '旺角',
  '何以',

In [21]:
# extract title words list
title_words_list = []
for value in author2title.values():
    title_words_list.append(value)
    
title_words_list

[['一起', 'mat'],
 ['做到',
  '一起',
  '體系',
  '到底',
  '分享',
  '運作',
  'ai',
  '建議',
  '什麼',
  'mat',
  '洗錢',
  'ptt',
  '必須',
  '平台',
  '這幾天',
  '區塊',
  '一點',
  '目前',
  '思考',
  '體系前',
  '看法',
  '模式',
  '權重',
  'matters',
  '心得',
  '可能',
  '可以',
  '生態'],
 ['今天',
  '转载',
  '何去',
  '談科學',
  '這篇',
  '北京',
  '回到',
  '绿皮',
  '守门人',
  '戰略',
  '比技藝',
  '港澳',
  '怎麼',
  '一封信',
  '劉蘇里',
  '介紹',
  '朋友',
  '六年',
  '两家',
  '一個',
  '告别',
  '尋找',
  '持續',
  '陌生人',
  '兇險',
  '時代',
  '往事',
  '看過',
  '所有',
  '之前',
  '局勢',
  '称为',
  '之一',
  '八旗',
  '信息',
  '李宗盛',
  'taxi',
  '襲警',
  '還能',
  '台大',
  '核试验',
  '一天',
  '反送',
  '以前',
  '广场',
  '獨立',
  '國家',
  '小时',
  'newsroom',
  '那些',
  '江雪',
  '學到',
  '墮落',
  '之間',
  '新聞',
  '哲學',
  '偶遇',
  '蒼白',
  '重要',
  '號角',
  '北大',
  '筆記',
  '意味著',
  '抗命',
  '火车',
  '中運動',
  '见文',
  '因何',
  '岳昕',
  '文摘',
  '三角地',
  '升級',
  '每个',
  '預示',
  '金正恩',
  '滿字',
  '合輯',
  '元朗',
  '還認',
  '下的粵',
  '空間',
  '書摘',
  '失敗',
  '變化',
  '需要',
  '李志德',
  '旺角',
  '何以',
  '轉載',
  '港大',
  '頭的',


## Word2Vec

In [22]:
model = word2vec.Word2Vec(title_words_list, seed=1, size=10, min_count=5)

In [23]:
model['香港']
model['香港'].tolist()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


[0.07495572417974472,
 -0.014911782927811146,
 0.034875646233558655,
 0.0021313950419425964,
 -0.06919337809085846,
 0.0030168460216373205,
 0.021082645282149315,
 -0.06736786663532257,
 0.03289094567298889,
 -0.028241055086255074]

In [24]:
model.most_similar('香港')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('性侵', 0.8694288730621338),
 ('一天', 0.8594504594802856),
 ('青春', 0.8537033796310425),
 ('創作', 0.8402427434921265),
 ('網路', 0.8317319750785828),
 ('我們', 0.8315181136131287),
 ('時代', 0.8183388113975525),
 ('台北', 0.7962713241577148),
 ('抗爭', 0.7952879667282104),
 ('為何', 0.7900763750076294)]

## kmeans cluster

In [25]:
all_words_list = set()
for i in title_words_list:
    all_words_list.update(i)
len(all_words_list)

10998

In [26]:
df = []
for i in all_words_list:
    if i in model:
        tmp = []
        tmp.append(i)
        tmp.extend(model[i].tolist())
        df.append(tmp)
df = pd.DataFrame(df)
df

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


,0,1,2,3,4,5,6,7,8,9,10
0,,0.019661,0.035942,0.042210,-0.016556,0.004947,-0.004879,-0.024887,-0.012471,-0.032538,-0.035181
1,保護,-0.040008,0.005325,-0.015245,0.037100,-0.038384,0.027763,-0.004255,0.026561,-0.021412,-0.033441
2,資訊,0.007950,0.009058,0.005752,-0.015228,-0.058272,-0.039198,0.018018,-0.022173,-0.019686,-0.024750
3,大陆,0.021827,0.016049,-0.004423,0.029867,-0.001096,0.071981,-0.037379,-0.036275,0.037231,0.024684
4,教學,0.014832,0.043400,-0.046776,-0.024085,-0.057129,0.045364,0.052278,0.013457,0.005296,0.016044
5,事件,0.093693,0.036749,-0.033238,-0.036718,-0.027460,0.027834,0.016480,-0.016210,-0.008550,-0.018665
6,之前,-0.031140,-0.012927,-0.038390,0.007454,-0.009647,-0.006353,0.032430,0.018769,0.049641,0.009754
7,歧視,0.024036,0.047130,0.020153,0.009469,-0.006877,0.020104,-0.034201,-0.045359,0.030130,-0.052203
8,轉型,-0.016674,-0.037740,-0.027446,-0.032200,-0.005050,-0.011216,0.002280,-0.000305,-0.039188,-0.028149
9,反送,0.039367,0.028864,-0.026976,0.003449,0.002068,0.024142,0.043554,-0.054159,-0.027202,0.024617


In [28]:
kmeans_ = KMeans(n_clusters = 10, random_state=0).fit(df.iloc[:,1:].values)

In [29]:
Counter(kmeans_.labels_)

Counter({2: 84, 6: 71, 9: 77, 7: 87, 4: 60, 8: 82, 5: 69, 1: 64, 3: 62, 0: 74})

In [30]:
model.similar_by_vector(kmeans_.cluster_centers_[1])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython kernel.


[('政治', 0.9301615953445435),
 ('關於', 0.9067676067352295),
 ('民族', 0.9054371118545532),
 ('內容', 0.9024325013160706),
 ('反思', 0.8811845779418945),
 ('數據', 0.8718485832214355),
 ('未來', 0.840681254863739),
 ('什麼樣', 0.837622880935669),
 ('责任', 0.8370625972747803),
 ('女性', 0.8223834037780762)]

In [31]:
model.similar_by_vector(kmeans_.cluster_centers_[2])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython kernel.


[('网络', 0.8256130218505859),
 ('立法', 0.8141503930091858),
 ('共享', 0.8060262799263),
 ('市場', 0.8057880997657776),
 ('沒有', 0.77424156665802),
 ('產品', 0.7719771862030029),
 ('更新', 0.7694795727729797),
 ('小說', 0.7675549387931824),
 ('學生', 0.7670901417732239),
 ('社会主义', 0.7641985416412354)]

### Top authors in favor of certain topics

In [32]:
author_description_list = []
# for word_vec in model.similar_by_vector(kmeans_.cluster_centers_[1]):
# for word_vec in model.most_similar('香港'):

for title in title_words_list:
    if '香港' in title:
#             use_count += 1
        author = list(author2title.keys())[title_words_list.index(title)]
        target_temp = str(df_authors.loc[df_authors['displayName'] == author,'description'])
        target = target_temp.split()[1]
        if target not in author_description_list:
            author_description_list.append(target)
#             print(df_authors[df_authors['displayName'] == author]['description'])

author_description_list      

['Name:',
 '副業是在香港中文大學教書，主業是玩貓。',
 'LikeCoin、oice、Lakoo創辦人。\\n地球人，信仰民主自由多元。\\n左而不膠。人...',
 '政治哲學教師。',
 '特別關注政治哲學與女性主義問題。',
 '香港人，職業學生，城市青年，近期興趣為港台農業、城鄉變遷、醫療系統/旅遊、都市人對身心靈的追...',
 '學展，政治工作者。台灣中壢人，是七年級也是90後，關注數位產業、台灣政治、媒體、兩岸議題。',
 '香港民間法律評論網站「法夢',
 '浸大學者，香港筆會執委。專注受眾狹隘的比較政治和社運研究，但深信知識分子在公共領域有角色。',
 '小老百姓，Internet',
 '独立策展人，注重艺术的社会责任，推动“艺术成为媒体”',
 '媒體從業者，專欄作者，你港人民感情傷害專家。',
 '資淺記者，人類學學徒。',
 '香港藝術家、學術人、策展人和作者。研究、創作與關注的課題包括當代城市研究、當代藝術、公共文化...',
 '邱林川\\n香港中文大學教授，香港平台合作社運動PCCHK創會成員。',
 '不務正業的學渣，世界主義者',
 '智庫「立言香港」召集人，前任香港中文大學學生會會長，現時從事商業和公民社會交流的工作，並在香...',
 '倫敦大學亞非學院博士研究生，從政治經濟視角研究法律文化和大專法律教育；前香港大專講師、前香港...',
 '七十後大學講師，十七歲入行編採，前香港曲棍球代表，編導香港電台十部外判紀錄片單元劇，首部電影...',
 '來自香港的書蟲，留意東南亞事務及共享經濟。在學印尼文。',
 '「區塊勢」創辦人，內向的文字表演者。化繁為簡、濃縮知識的狂熱者。',
 '香港粵語片研究會成員。前《信報》「電影講座」專欄作者，文章現散見於「香港',
 '学术工作者、无名作家。',
 '职业经理人，电子电脑技术专家。目前是专业的生活家。',
 '独立媒体，非营利性质，我们关注环境、教育、性/别、精神健康等公共议题，提供负责的纪实性内容。...',
 '香港文化評論人、藝評人及詩人，香港中文大學講師，「四圍講古」發起人，國際演藝評論家協會（香港...',
 '電影朝聖，一個值得發掘的角度與方式去認識我城。我是作者Gary',
 '教育及文字工作者，喜歡說故事，也是說書人。香港中文大學新聞與傳播學士，英文應